<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 Guia Prático de Fine-Tuning: Llama 3 8B com QLoRA

Olá! Este notebook é seu guia passo a passo para o mundo do fine-tuning de LLMs. Como estudante de Engenharia de Computação, é crucial entender não apenas *como* fazer, mas *por que* as técnicas funcionam.

### As Estratégias de Fine-Tuning

1.  **Full Fine-Tuning**:
    * **O que é?** Ajustar *todos* os bilhões de pesos do modelo.
    * **Problema:** Requer uma quantidade massiva de VRAM (memória de GPU), sendo inviável para modelos de 8B em hardware comum ou no Colab. Um modelo de 8B com precisão total (32-bit) precisaria de `8B * 4 bytes = 32 GB` de VRAM só para ser carregado, sem contar a memória extra para o treinamento.

2.  **PEFT (Parameter-Efficient Fine-Tuning)**:
    * **O que é?** Uma família de técnicas que congela os pesos originais do LLM (que são 99.9% do total) e treina apenas um número minúsculo de novos parâmetros "adaptadores".
    * **Vantagem:** Redução drástica no uso de memória e aceleração do treino.

3.  **LoRA (Low-Rank Adaptation)**:
    * **A Estrela do PEFT.** A ideia é que a "atualização" dos pesos para uma nova tarefa pode ser representada por matrizes de baixo posto (muito menores). Em vez de modificar uma matriz de peso gigante `W`, adicionamos o resultado de duas matrizes pequenas, `A` e `B`, que são treináveis (`W' = W + B*A`).

4.  **QLoRA (Quantized LoRA)**:
    * **A Magia que nos permite rodar no Colab.** É uma otimização do LoRA que faz duas coisas geniais:
        1.  **Quantização:** Carrega o modelo principal (Llama 3 8B) com precisão reduzida (4-bit em vez de 16-bit), cortando o uso de memória em 4x.
        2.  **Adaptação LoRA:** Treina os adaptadores LoRA normalmente sobre esse modelo quantizado.

**Nosso objetivo hoje:** Vamos ensinar o `Meta Llama 3 8B Instruct` a responder a um comando em linguagem natural com um output em formato JSON bem estruturado, uma tarefa muito útil em engenharia de software.

In [1]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install trl
!pip install triton


In [13]:
from huggingface_hub import login
from google.colab import userdata

# Retrieve the HF_TOKEN from Colab secrets and login
hf_token = userdata.get("HF_TOKEN")
login(token=hf_token)

In [3]:
# Para este exemplo, não vamos usar um dataset externo.
# Vamos criar um pequeno dataset na mão para ensinar o modelo a estruturar saídas em JSON.

from datasets import Dataset

# Nosso objetivo é transformar um texto não estruturado sobre um usuário em um JSON.
instructions = [
    "Extraia o nome, a idade e a cidade de: 'O usuário João Silva tem 28 anos e mora em São Paulo.'",
    "Transforme em JSON os dados de: 'Ana Souza, 35 anos, residente do Rio de Janeiro.'",
    "Formate as seguintes informações: 'Carlos Pereira, de Curitiba, tem 42 anos.'"
]

outputs = [
    '{"nome": "João Silva", "idade": 28, "cidade": "São Paulo"}',
    '{"nome": "Ana Souza", "idade": 35, "cidade": "Rio de Janeiro"}',
    '{"nome": "Carlos Pereira", "idade": 42, "cidade": "Curitiba"}'
]

# O formato do prompt é crucial para modelos "instruct".
# O Mistral também usa um formato específico.
# Vamos formatar nossos dados nesse padrão.
formatted_data = []
for instruction, output in zip(instructions, outputs):
    # Mistral prompt format
    text = f"[INST]{instruction}[/INST]{output}"
    formatted_data.append({"text": text})

# Criando um objeto Dataset da Hugging Face
dataset = Dataset.from_dict({"text": [item["text"] for item in formatted_data]})

print("Exemplo de um item do dataset formatado:")
print(dataset[0]['text'])

Exemplo de um item do dataset formatado:
[INST]Extraia o nome, a idade e a cidade de: 'O usuário João Silva tem 28 anos e mora em São Paulo.'[/INST]{"nome": "João Silva", "idade": 28, "cidade": "São Paulo"}


In [40]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Carregando o modelo base
model_id = "google/gemma-2b"

# Configuração de quantização (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype="auto"
)

# Carregando o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


# Ativa otimizações para o treino com quantização
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

# Configuração dos adaptadores LoRA
lora_config = LoraConfig(
    r=8,  # "rank" da decomposição. Um valor maior treina mais parâmetros, mas pode levar a overfitting. 16 é um bom começo.
    lora_alpha=16, # Parâmetro de escala. A regra geral é que seja 2 * r.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # Common target modules for decoder models
    lora_dropout=0.05, # Dropout para regularização
    bias="none",
    task_type="CAUSAL_LM" # Tarefa de modelagem de linguagem causal
)

# Aplica o wrapper PEFT no nosso modelo
peft_model = get_peft_model(model, lora_config)

# Imprime o número de parâmetros treináveis para vermos a mágica do PEFT
peft_model.print_trainable_parameters()
# Você verá que o número de parâmetros treináveis é < 1% do total!

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/8.38G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/8.38G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/18.1M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

trainable params: 23,068,672 || all params: 4,213,577,728 || trainable%: 0.5475


In [41]:
# Célula 7: Execução do Treinamento
from transformers import TrainingArguments
from trl import SFTTrainer
import bitsandbytes

# Argumentos do treinamento
training_args = TrainingArguments(
    output_dir="./google/gemma-2b-json-finetune", # Diretório para salvar o modelo
    per_device_train_batch_size=1, # Batch size pequeno para caber na memória
    gradient_accumulation_steps=4, # Simula um batch size maior (1*4=4) para estabilizar o treino
    learning_rate=2e-4, # Taxa de aprendizado
    max_steps=100, # Número de passos de treino. Com um dataset pequeno, 100 é suficiente.
    logging_steps=10, # Logar o progresso a cada 10 passos
    fp16=True, # Usar precisão de 16-bit para o treino
    # bf16=True, # Uncomment this line if your GPU supports bfloat16
    push_to_hub=False,# Set to True to push your model to the Hugging Face Hub
)

# Criando o objeto Trainer
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset,
    args=training_args,
    max_seq_length=512, # Comprimento máximo da sequência
    tokenizer=tokenizer,
    dataset_text_field="text", # O campo do nosso dataset que contém o texto formatado
    # packing=True, # Uncomment this line to use packing (more efficient for short sequences)
)

# Inicia o treinamento
trainer.train()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:402: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 3}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: isaac052503 (isaac052503-unicamp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [langchain] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.981300
20,0.028000
30,0.022400
40,0.021500
50,0.021200
60,0.021200
70,0.021100
80,0.020900
90,0.020900
100,0.021100


TrainOutput(global_step=100, training_loss=0.11796499043703079, metrics={'train_runtime': 623.5432, 'train_samples_per_second': 0.641, 'train_steps_per_second': 0.16, 'total_flos': 320780840140800.0, 'train_loss': 0.11796499043703079, 'epoch': 100.0})

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [68]:
# Crie uma pasta no seu Drive para guardar os modelos (só precisa rodar uma vez)
!mkdir -p /content/drive/MyDrive/meus_modelos_ic/

# Copie a pasta do seu adaptador treinado para o Google Drive
!cp -r ./google/gemma-2b-json-finetune/ /content/drive/MyDrive/meus_modelos_ic/
print("Adaptador salvo com segurança no Google Drive!")

Adaptador salvo com segurança no Google Drive!


In [12]:
# Delete peft_model and clear cache to free up memory
# del peft_model
torch.cuda.empty_cache()

print("peft_model deleted and CUDA cache cleared.")

peft_model deleted and CUDA cache cleared.


In [ ]:
# Delete some libraries

# 🧬 Parte 2: Mesclando (Merging) seu Adaptador LoRA

Parabéns, você treinou um adaptador LoRA! No entanto, no estado atual, você tem duas partes: o modelo base gigante (Llama 3 8B) e seu pequeno adaptador LoRA. Para fazer uma inferência, você precisa carregar ambos.

A **mesclagem** é o processo de "fundir" os pesos do seu adaptador LoRA de volta aos pesos do modelo base. O resultado é um **único modelo autônomo** que já contém a sua especialização.

### Por que mesclar?

1.  **Simplificação de Deploy:** Em vez de gerenciar o modelo base + o adaptador, você distribui um único modelo. É muito mais simples para colocar em produção.
2.  **Performance de Inferência:** A inferência pode ser ligeiramente mais rápida, pois o modelo não precisa mais combinar dinamicamente os pesos do LoRA com os pesos base a cada passada. O cálculo `W' = W + B*A` já foi feito e "assado" no modelo.
3.  **Compartilhamento:** Permite que você compartilhe seu modelo fine-tuned completo na Hugging Face como um novo modelo, e não apenas como um adaptador.

Vamos ver como fazer isso na prática.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# Caminho para o adaptador salvo no seu Google Drive
adapter_path = "/content/drive/MyDrive/meus_modelos_ic/google/gemma-2b-json-finetune"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Célula 11: Carregando o Modelo Base e o Adaptador para Mesclagem
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# --- Carregando o Modelo Base ---
# Desta vez, vamos carregar o modelo em precisão reduzida (4-bit)
# para a mesclagem, caso a versão de 16-bit exija muita memória.

model_id = "google/gemma-2b"

# Configuração de quantização (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # Use the 4-bit quantization config
    device_map="auto", # Explicitly set device_map to auto
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# --- Carregando o Adaptador PEFT ---
# Apontamos para o diretório onde o `trainer` salvou nosso adaptador LoRA.
peft_model = PeftModel.from_pretrained(base_model, adapter_path)

print("Modelo base e adaptador carregados com sucesso.")

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.46 GiB. GPU 0 has a total capacity of 14.74 GiB of which 220.12 MiB is free. Process 713075 has 14.52 GiB memory in use. Of the allocated memory 12.83 GiB is allocated by PyTorch, and 1.58 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Célula 12: Executando a Mesclagem e Testando o Modelo Final

# Este é o comando mágico que funde os pesos!
# Ele "descarrega" o wrapper PEFT e retorna um modelo transformers padrão.
merged_model = peft_model.merge_and_unload()

print("Mesclagem concluída!")

# --- Teste do Modelo Mesclado ---
# Agora, podemos usar este `merged_model` como qualquer outro modelo da Hugging Face.
# Note que não precisamos mais do objeto `peft_model` aqui.

from transformers import pipeline

# O mesmo prompt de teste que usamos antes
test_prompt = tokenizer.apply_chat_template(
    [{"role": "user", "content": "Extraia as informações de 'Mariana Lima, 25 anos, de Salvador.' em formato JSON."}],
    tokenize=False,
    add_generation_prompt=True
)

# Criamos um pipeline com o nosso NOVO modelo mesclado
merged_pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer)

# Executamos a inferência
merged_output = merged_pipe(test_prompt, max_new_tokens=50, do_sample=False)

print("\n--- Resposta do Modelo Mesclado e Autônomo ---")
print(merged_output[0]['generated_text'])

# O resultado deve ser idêntico ao do modelo com o adaptador PEFT,
# provando que a mesclagem foi um sucesso!

In [ ]:
# Célula 13: Salvando o Modelo Mesclado Completo para o Disco

# Agora você tem um modelo completo. Vamos salvá-lo.
# Este diretório conterá todos os arquivos necessários para carregar o modelo
# sem precisar do código do fine-tuning ou dos adaptadores.

output_merged_dir = "./google/gemma-2b-json-finetune" # Updated output directory

merged_model.save_pretrained(output_merged_dir)
tokenizer.save_pretrained(output_merged_dir)

print(f"Modelo mesclado completo salvo em: {output_merged_dir}")
# A partir daqui, você poderia carregar este modelo com um simples:
# AutoModelForCausalLM.from_pretrained("./google/gemma-2b-json-finetune")

# 🧬🧪 Tópico Avançado: Mesclando Múltiplos Modelos com `mergekit`

E se você tivesse treinado **vários adaptadores** LoRA?
* Um para gerar JSON (o que fizemos).
* Outro para ser um especialista em programação Python.
* Um terceiro para escrever de forma criativa.

Você pode mesclá-los para criar um "super-modelo" que faz tudo isso. A função `.merge_and_unload()` não é ideal para isso. A ferramenta padrão da comunidade é a **`mergekit`**.

`mergekit` funciona com um arquivo de configuração YAML onde você especifica os modelos e a estratégia de mesclagem.

**Exemplo de um arquivo `config.yml` para `mergekit`:**

```yaml
# Define o modelo base que servirá de alicerce
base_model: meta-llama/Meta-Llama-3-8B-Instruct

# Lista os adaptadores LoRA que você quer mesclar sobre o base
slices:
  - sources:
      # Nosso primeiro adaptador treinado
      - model: ./llama3-8b-json-finetune
        # Damos um peso positivo para sua contribuição
        positive_prompt: '{"nome": "João", "idade": 30, "cidade": "Qualquer"}'
      # Imagine um segundo adaptador treinado para Python
      - model: ./llama3-8b-python-coder-finetune
        positive_prompt: 'def hello_world(): print("Hello, World!")'

# Define o método de mesclagem (SLERP é geralmente melhor que a média linear)
merge_method: ties
# Define a precisão dos números no modelo final
dtype: bfloat16